In [1]:
from dotenv import load_dotenv
import os 
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser

load_dotenv('env', override=True)
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
END_POINT=os.getenv('END_POINT')
MODEL_NAME=os.getenv('MODEL_NAME')
print(AZURE_OPENAI_API_KEY[:10])
print(MODEL_NAME)

AZURE_OPENAI_EMB_API_KEY = os.getenv('AZURE_OPENAI_EMB_API_KEY')
EMB_END_POINT=os.getenv('EMB_END_POINT')
EMB_MODEL_NAME=os.getenv('EMB_MODEL_NAME')

os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGSMITH_API_KEY')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_TRACING_V2'] = 'true' #true, false
os.environ['LANGCHAIN_PROJECT'] = 'LANG'

if os.getenv('LANGCHAIN_TRACING_V2') == "true":
    print('랭스미스로 추적 중입니다 :', os.getenv('LANGSMITH_API_KEY')[:10])

43b13g4OZS
gpt-4.1-mini
랭스미스로 추적 중입니다 : lsv2_pt_24


# Output Parser


In [2]:
email_conversation = """From: 박지훈 (jihoon.park@neobikedistribution.kr)
To: 최민서 (minseo.choi@teddyinternational.me)
Subject: ‘AURORA Pro’ 국내 유통 파트너십 제안 및 미팅 일정 협의

안녕하세요, 최민서 팀장님.

네오바이크디스트리뷰션의 박지훈 이사입니다. 최근 귀사의 신형 e-바이크 “AURORA Pro” 출시 소식을 접했고, 한국 내 유통 파트너십 가능성을 검토하고자 연락드립니다. 당사는 전국 120개 오프라인 딜러 네트워크와 자사몰(월 UV 35만)을 운영 중입니다.

혹시 아래의 내용에 대해서 답변을 해 주실 수 있을까요?

우선 제품 문서가 필요합니다. 상세 스펙시트(모터 출력, 배터리 용량/셀 타입, 1회 충전 주행거리, 충전 시간, 프레임 소재/무게), 보증 정책/AS 프로세스가 적혀있으면 좋겠습니다.

인증/규격은 어떤가요? KC·CE·UL 보유하고 계신가요?

가격은 얼마인지. 그리고 최소주문수량은 어느정도 되어야 하는지요?

관련 이미지나 홍보자료가 있다면 같이 첨부해 주시면 좋겠습니다.

가능한 빠른시일내에 미팅 일정을 잡고 싶습니다.
아래 두 일정 중 가능하신 시간 알려주시면 감사하겠습니다. 예상 시간은 30~45분 정도입니다.

2025년 9월 16일(화) 10:00 KST | 장소: 귀사 본사 또는 온라인(Zoom)
2025년 9월 18일(목) 14:00 KST | 동일


검토 부탁드립니다. 가능 일정 회신 주시면 즉시 캘린더 초대 발송하겠습니다.
감사합니다.

박지훈 이사
네오바이크디스트리뷰션
M. 010-2345-6789 | E. jihoon.park@neobikedistribution.kr

서울시 성동구 ○○로 12, 6F
"""

In [3]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=END_POINT,  
    azure_deployment=MODEL_NAME,          
    api_version="2024-12-01-preview",
    temperature=0.2,
)

print(llm.invoke(f"다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}").content)

다음은 이메일의 중요한 내용 요약입니다:

- 발신자: 박지훈 이사 (네오바이크디스트리뷰션)
- 수신자: 최민서 팀장 (테디인터내셔널)
- 목적: 신형 e-바이크 ‘AURORA Pro’의 한국 내 유통 파트너십 제안 및 미팅 일정 협의
- 네오바이크디스트리뷰션 현황: 전국 120개 오프라인 딜러 네트워크, 자사몰 월 UV 35만
- 요청사항:
  - 제품 상세 문서(스펙시트: 모터 출력, 배터리 용량/셀 타입, 1회 충전 주행거리, 충전 시간, 프레임 소재/무게)
  - 보증 정책 및 AS 프로세스
  - 인증/규격 보유 여부(KC, CE, UL)
  - 가격 및 최소주문수량
  - 관련 이미지 및 홍보자료
- 미팅 일정 제안:
  - 2025년 9월 16일(화) 10:00 KST
  - 2025년 9월 18일(목) 14:00 KST
  - 장소: 귀사 본사 또는 온라인(Zoom)
  - 예상 소요 시간: 30~45분
- 회신 요청: 가능한 일정 회신 시 캘린더 초대 발송 예정


## Pydantic

- 구조를 지정하고 검증하는 게 포함된 클래스로 “형식 지시 + 파싱 + Pydantic 검증”으로 안정적인 구조로 출력할 수 있음

In [10]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람") #필드명, 타입, 설명, 초기값까지 설정 가능
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

parser = PydanticOutputParser(pydantic_object=EmailSummary)
format_instructions = parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [12]:
# 2) 프롬프트 (형식 지시를 반드시 주입!)
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "다음 이메일 원문에서 보낸 사람/이메일/제목/본문 요약/미팅 시간을 추출해. "
     "모르면 빈 문자열로 두고 추측하지 마."),
    ("human",
     "출력은 아래 형식만 지켜 JSON으로만 내보내.\n{format}\n\n"
     "=== 이메일 원문 시작 ===\n{email_raw}\n=== 이메일 원문 끝 ===")
]).partial(format=format_instructions)

for p in prompt:
    print(p)

('name', None)
('input_variables', ['email_raw'])
('optional_variables', [])
('input_types', {})
('output_parser', None)
('partial_variables', {'format': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "tit

프롬프트의 partial_variables에 dataclass가 해체되어 문자열로 입력되었습니다.

In [16]:
# 파서 없이 출력력
chain_wo_parser = prompt | llm
output = chain_wo_parser.invoke({"email_raw": email_conversation}).content
print(type(output))
print(output)

<class 'str'>
{
  "person": "박지훈",
  "email": "jihoon.park@neobikedistribution.kr",
  "subject": "‘AURORA Pro’ 국내 유통 파트너십 제안 및 미팅 일정 협의",
  "summary": "박지훈 이사는 네오바이크디스트리뷰션을 대표하여 AURORA Pro e-바이크의 국내 유통 파트너십 가능성을 문의하며, 제품 상세 스펙, 보증 정책, 인증, 가격, 최소주문수량, 홍보자료 요청과 함께 미팅 일정을 조율하고자 함.",
  "date": "2025년 9월 16일(화) 10:00 KST 또는 2025년 9월 18일(목) 14:00 KST"
}


In [17]:
# 세가지 러너블을 하나의 체인으로 묶습니다
chain = prompt | llm | parser

In [18]:
result = chain.invoke({"email_raw": email_conversation}) #비어있는 email_raw에 이메일 원문을 삽입
print(type(result))
print(result)

<class '__main__.EmailSummary'>
person='박지훈' email='jihoon.park@neobikedistribution.kr' subject='‘AURORA Pro’ 국내 유통 파트너십 제안 및 미팅 일정 협의' summary='박지훈 이사는 ‘AURORA Pro’ 신형 e-바이크의 국내 유통 파트너십 가능성을 검토하고자 연락하며, 제품 문서, 인증, 가격, 최소주문수량, 홍보자료 요청과 함께 빠른 미팅 일정을 희망함.' date='2025년 9월 16일(화) 10:00 KST 또는 2025년 9월 18일(목) 14:00 KST'


In [19]:
for k , v in dict(result).items():
    print('----------------------------')
    print(f'[{k}]')
    print(v)

----------------------------
[person]
박지훈
----------------------------
[email]
jihoon.park@neobikedistribution.kr
----------------------------
[subject]
‘AURORA Pro’ 국내 유통 파트너십 제안 및 미팅 일정 협의
----------------------------
[summary]
박지훈 이사는 ‘AURORA Pro’ 신형 e-바이크의 국내 유통 파트너십 가능성을 검토하고자 연락하며, 제품 문서, 인증, 가격, 최소주문수량, 홍보자료 요청과 함께 빠른 미팅 일정을 희망함.
----------------------------
[date]
2025년 9월 16일(화) 10:00 KST 또는 2025년 9월 18일(목) 14:00 KST


In [20]:
print(result.person)
print(result.email)

박지훈
jihoon.park@neobikedistribution.kr


### Structured_output

정해진 구조(JSON, Pydantic등)에 맞게 애초부터 모델에게 이런 구조로 출력하도록 전달해서 강제할 수 있습니니다.

하지만 완전한 안정성 보장은 아니고 드물게 오류가 날 가능성이 있음

그래도 PydanticOutputParser보다는 조금 더 나은편

In [21]:
structured_llm = llm.with_structured_output(EmailSummary) #<-- 모델과 출력 파서가 하나로 결합

prompt = ChatPromptTemplate.from_messages([
    
    ("system", "이메일에서 핵심 정보를 추출해. 모르면 빈 문자열로 두고, 추측하지 마."),
    ("human", "다음 이메일 원문:\n{email_raw}")
])

chain = prompt | structured_llm

result = chain.invoke({"email_raw": email_conversation})
print(type(result))
print(result)

<class '__main__.EmailSummary'>
person='박지훈' email='jihoon.park@neobikedistribution.kr' subject='‘AURORA Pro’ 국내 유통 파트너십 제안 및 미팅 일정 협의' summary="박지훈 이사는 최민서 팀장에게 'AURORA Pro' e-바이크의 국내 유통 파트너십 가능성을 문의하며, 제품 상세 스펙시트, 보증 정책, 인증 현황, 가격 및 최소 주문 수량, 홍보자료 요청과 함께 미팅 일정을 제안함." date='2025-09-15'


### outputFixingParser

모델에게 단순히 json으로 요청했을 때 잘 되면 다행이지만 하나라도 char가 깨지만 json으로 변경할 수 없다.

OutputFixingParser는 LLM 출력이 스키마(예: JSON)와 맞지 않아 파싱에 실패했을 때,    
후처리 단계에서 LLM을 한 번 더 호출해 형식/필드/문자 깨짐 등을 교정하여 유효한 결과로 복구해 주는 파서 래퍼입니다.    
안정성이 올라가지만, 추가 토큰 비용과 지연이 발생합니다.    

In [24]:
from langchain.output_parsers import OutputFixingParser
fixing_parser = OutputFixingParser.from_llm(llm=llm, parser=parser)
result = (prompt | llm | fixing_parser).invoke({"email_raw": email_conversation})
print(type(result))
print(result)

<class '__main__.EmailSummary'>
person='박지훈' email='jihoon.park@neobikedistribution.kr' subject='‘AURORA Pro’ 국내 유통 파트너십 제안 및 미팅 일정 협의' summary="신형 e-바이크 'AURORA Pro'의 국내 유통 파트너십 검토 요청과 제품 상세 스펙, 보증 정책, 인증 보유 여부, 가격 및 최소 주문 수량, 홍보자료 요청. 미팅 일정으로 2025년 9월 16일 10:00 KST 또는 9월 18일 14:00 KST를 제안하며, 장소는 귀사 본사 또는 온라인(Zoom)으로 예정. 예상 미팅 시간은 30~45분." date='2025-09-16 10:00 KST, 2025-09-18 14:00 KST'


In [23]:
dict(result)

{'person': '박지훈',
 'email': 'jihoon.park@neobikedistribution.kr',
 'subject': '‘AURORA Pro’ 국내 유통 파트너십 제안 및 미팅 일정 협의',
 'summary': '‘AURORA Pro’ 제품의 상세 스펙시트, 보증 정책 및 AS 프로세스, 인증/규격 보유 여부, 가격 및 최소주문수량, 관련 이미지 및 홍보자료 요청. 미팅 일정으로 2025년 9월 16일 10:00 KST 또는 2025년 9월 18일 14:00 KST 제안, 예상 소요 시간 30~45분.',
 'date': '2025-09-16 10:00 KST, 2025-09-18 14:00 KST'}